In [3]:
import wave
from audioop import add, mul
from warnings import warn

In [4]:
def input_wave(filename,frames=10000000): #10000000 is an arbitrary large number of frames
    with wave.open(filename,'rb') as wave_file:
        params=wave_file.getparams()
        audio=wave_file.readframes(frames)  
        if params.nchannels!=1:
            raise Exception("The input audio should be mono for these examples")
    return params, audio

#output to file so we can use ipython notebook's Audio widget
def output_wave(audio, params, stem, suffix):
    #dynamically format the filename by passing in data
    filename=stem.replace('.wav','_{}.wav'.format(suffix))
    with wave.open(filename,'wb') as wf:
        wf.setparams(params)
        wf.writeframes(audio)
        wf.close()

In [12]:
TWDY_params, TWDY_bytes = input_wave('/Volumes/S200408/backup/S190813/Coding/BC_REPO/this-will-destroy-you/chop_wav/wav_0_10_.wav')

In [22]:
def delay(audio_bytes,params,offset_ms):
    """version 1: delay after 'offset_ms' milliseconds"""
    #calculate the number of bytes which corresponds to the offset in milliseconds
    offset= params.sampwidth*offset_ms*int(params.framerate/1000)
    #create some silence
    beginning= b'\0'*offset
    #remove space from the end
    end= audio_bytes[:-offset]
    return add(audio_bytes, beginning+end, params.sampwidth)




#new delay function with factor
def delay_2(audio_bytes,params,offset_ms,factor=1):
    """version 2: delay after 'offset_ms' milliseconds amplified by 'factor'"""
    #calculate the number of bytes which corresponds to the offset in milliseconds
    offset= params.sampwidth*offset_ms*int(params.framerate/1000)
    #create some silence
    beginning= b'\0'*offset
    #remove space from the end
    end= audio_bytes[:-offset]
    #multiply by the factor
    multiplied_end= mul(audio_bytes[:-offset],params.sampwidth,factor)
    return add(audio_bytes, beginning+ multiplied_end, params.sampwidth)



def delay_3(audio_bytes,params,offset_ms,factor=1,num=1):
    """version 3: 'num' delays after 'offset_ms' milliseconds amplified by 'factor'"""
    if factor>=1:
        warn("These settings may produce a very loud audio file. \
             Please use caution when listening")
    #calculate the number of bytes which corresponds to the offset in milliseconds
    offset=params.sampwidth*offset_ms*int(params.framerate/1000)
    #add extra space at the end for the delays
    delayed_bytes=audio_bytes
    for i in range(num):
        #create some silence
        beginning = b'\0'*offset*(i+1)
        #remove space from the end
        end = audio_bytes[:-offset*(i+1)]
        #multiply by the factor
        multiplied_end= mul(end,params.sampwidth,factor**(i+1))
        delayed_bytes= add(delayed_bytes, beginning+multiplied_end, params.sampwidth)
    return delayed_bytes



def delay_4(audio_bytes,params,offset_ms,factor=1,num=1):
    """version 4: 'num' delays after 'offset_ms' milliseconds amplified by 'factor' 
    with additional space"""
    if factor>=1:
        warn("These settings may produce a very loud audio file. \
              Please use caution when listening")
    #calculate the number of bytes which corresponds to the offset in milliseconds
    offset=params.sampwidth*offset_ms*int(params.framerate/1000)
    #add extra space at the end for the delays
    audio_bytes=audio_bytes+b'\0'*offset*(num)
    #create a copy of the original to apply the delays
    delayed_bytes=audio_bytes
    for i in range(num):
        #create some silence
        beginning = b'\0'*offset*(i+1)
        #remove space from the end
        end = audio_bytes[:-offset*(i+1)]
        #multiply by the factor
        multiplied_end= mul(end,params.sampwidth,factor**(i+1))
        delayed_bytes= add(delayed_bytes, beginning+multiplied_end, params.sampwidth)
    return delayed_bytes





In [26]:
output_wave(delay(t_bytes, t_params, 250), t_params, 't_0_10.wav', 'delay_250ms')

In [25]:
t_params, t_bytes = input_wave('/Volumes/S200408/backup/S190813/Coding/wav_utilities/wav_utilities/test_stretch.wav')
output_wave(delay_2(t_bytes, t_params, offset_ms=1000, factor=0.5), t_params, 't_0_10.wav', 'delay_1000ms_0.5f')

In [29]:
output_wave(delay_3(t_bytes, t_params, offset_ms=250, factor=0.5, num=11), t_params, 't_0_10.wav', 'delay_250ms_0.5f_11d')

In [30]:
output_wave(delay_4(t_bytes, t_params, offset_ms=100, factor=0.5, num=5), t_params, 't_0_10_.wav', 'delay_100ms_0.5f_5d')

In [1]:
from delay import *

In [3]:
t_params, t_bytes = input_wave('/Volumes/S200408/backup/S190813/Coding/wav_utilities/wav_utilities/test_stretch.wav')
output_wave(delay_4(t_bytes, t_params, offset_ms=100, factor=0.5, num=5), t_params, 't_0_10_.wav', 'delay_100ms_0.5f_5d')

In [4]:
import reverse

In [6]:
wav = '/Volumes/S200408/backup/S190813/Coding/BC_REPO/this-will-destroy-you/chop_wav/wav_0_10_.wav'
new_wav = 'R.wav'
reverse.reverse(wav, new_wav)

In [1]:
import volume

In [5]:
wav = '/Volumes/S200408/backup/S190813/Coding/wav_utilities/wav_utilities/R.wav'
new_wav = 'louder.wav'
volume.adjust_volume(wav, new_wav, 'decrease', 4)

In [12]:
from scipy.io.wavfile import read, write
from scipy.fftpack import rfft, irfft
import numpy as np
wav = '/Volumes/S200408/backup/S190813/Coding/wav_utilities/wav_utilities/louder.wav'
rate, input = read(wav)
transformed = rfft(input) 
output = irfft(transformed)
write('smaller.wav', rate, output.astype(input.dtype))